# IoT Challenge - Geolocalization

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from geopy.distance import vincenty

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

In [3]:
# load train and test data
df_mess_train = pd.read_csv('mess_train_list.csv') # train set
df_mess_test = pd.read_csv('mess_test_list.csv') # test set
pos_train = pd.read_csv('pos_train_list.csv') # position associated to train set

## Data exploration [Théo]

## Features Engineering [Mathieu]

### One-Hot (Teacher ++ / Mean / RSSI / etc.. )

### Usage of centroid

#### Standard centroid

#### Weighted centroid

## Outliers processing I [André]

## Model [Mathieu / Matyas / André]

### First raw predictions

### Outliers processing II

### Fine-tunning / Model selection

### Blending

## Test analysis [André] 